# **Image to text**

## Phi 3.5 Vision

In [ ]:
from IPython.display import Markdown, display
from PIL import Image
import requests
from transformers import AutoModelForCausalLM, AutoProcessor
import gc
import torch

class Img2Text:
    def __init__(self, device):
        self.device = device
        self.model_id = "microsoft/Phi-3.5-vision-instruct"
        self.model = AutoModelForCausalLM.from_pretrained(self.model_id,
                                                          device_map=self.device,
                                                          trust_remote_code=True,
                                                          torch_dtype="auto",
                                                          _attn_implementation='eager'
                                                        )
        self.processor = AutoProcessor.from_pretrained(self.model_id, trust_remote_code=True)
        self.messages = [{"role": "user", "content": "<|image_1|> Describe the image in Details"}]
        self.prompt = self.processor.tokenizer.apply_chat_template(self.messages, tokenize=False, add_generation_prompt=True)

    def run(self, image, messages=False):
        # Load the image using PIL if it is a path or URL
        if isinstance(image, str):
            image = Image.open(image).convert("RGB")
        # If image is not a list, make it a list containing the image
        if not isinstance(image, list):
            image = [image]
        if messages:
            self.prompt = self.processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

        inputs = self.processor(self.prompt, image, return_tensors="pt").to(self.device)
        generation_args = {"max_new_tokens": 1000, "temperature": 1, "do_sample": False}
        generate_ids = self.model.generate(**inputs, eos_token_id=self.processor.tokenizer.eos_token_id, **generation_args)
        generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
        response = self.processor.batch_decode(generate_ids,
                                  skip_special_tokens=True,
                                  clean_up_tokenization_spaces=False)[0]
        del inputs
        del generate_ids
        torch.cuda.empty_cache()
        return response

    def cleanup(self):
        del self.model
        del self.processor
        torch.cuda.empty_cache()
        gc.collect()

In [ ]:
Imgt2Text = Img2Text("cuda")
image = "/content/360_F_503362352_Q4oLkiACXRUv0uKVIonzo525a78Jf6d2.jpg"
Img2Text.run(image)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/image_processing_auto.py:513: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


"The image captures a heartwarming scene of a family of foxes in their natural habitat. The mother fox, with her rich brown fur, stands tall and proud, her gaze fixed on the camera. She is flanked by her two cubs, their fur a lighter shade of brown, their eyes wide with curiosity and wonder. The cubs are exploring the grassy terrain, their small paws leaving imprints on the soft earth. The background is a blur of green, suggesting a dense forest or a grassy field, providing a safe and secluded environment for the family. The image is taken from a low angle, making the foxes appear larger and more majestic. The lighting is soft and natural, casting a warm glow on the scene and highlighting the intricate details of the foxes' fur and features. The image does not contain any text."

In [ ]:
Img2Text.cleanup()

# **English to Arabic Translation**

## llama3

In [ ]:
!pip install groq -q
from groq import Groq

class Trans2Arabic:
    def __init__(self):
        self.api_key = "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"
        self.inst_prompt = """You are a skilled translator with extensive experience in English and Arabic translations.
                            You possess a deep understanding of the linguistic, cultural, and contextual nuances essential for accurate and effective translation between these languages. Highly motivated and detail-oriented, you are committed to delivering translations that maintain the integrity and intent of the original text.
                            Your role is crucial in ensuring clear and precise communication in our multilingual system.
                            Do not add anything other than the description"""

    def run(self, text):
        client = Groq(api_key=self.api_key)
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": f"{self.inst_prompt} text to translate: {text}",
                }
            ],
            model="llama3-70b-8192",
            temperature=0,
        )

        return chat_completion.choices[0].message.content


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 3.1 MB/s eta 0:00:00


In [ ]:
translator = Trans2Arabic()
text = "The image captures a heartwarming scene of a family of foxes in their natural habitat. The mother fox, with her rich brown fur, stands tall and proud, her gaze fixed on the camera. She is flanked by her two cubs, their fur a lighter shade of brown, their eyes wide with curiosity and wonder. The cubs are exploring the grassy terrain, their small paws leaving imprints on the soft earth. The background is a blur of green, suggesting a dense forest or a grassy field, providing a safe and secluded environment for the family. The image is taken from a low angle, making the foxes appear larger and more majestic. The lighting is soft and natural, casting a warm glow on the scene and highlighting the intricate details of the foxes' fur and features. The image does not contain any text."

In [ ]:
translated_text = translator.run(text)
print(translated_text)

تُصوّر الصورة مشهداً رومانسياً لعائلة الثعالب في بيئتها الطبيعية. تقف الثعلبة الأم، ذات الفراء الغني باللون البني، منتصبةً وفخورةً، نظرتها ثابتةً على الكاميرا. وتحيط بها صغيراها، فرائهما أفتح لوناً من البني، عيناهما واسعتان بالفضول والدهشة. ويستكشف الصغيران التضاريس العشبية، أقدامهما الصغيرة تترك آثاراً على الأرض الناعمة. الخلفية مُبهَمة باللون الأخضر، مما يوحي بوجود غابة كثيفة أو ميدان عشبي، مما يوفر بيئة آمنة ومُ


# Text to Speech TTS Arabic

## TTs using Arabic tts

## TTs using xtts

In [ ]:
!pip install TTS
import torch
from TTS.api import TTS

class T2S_AR:
    def __init__(self, clone_voice_path, device="cuda:0"):
        self.clone_voice_path = clone_voice_path
        self.out_path = "output.wav"
        self.device = device
        self.tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(self.device)

    def run(self, text):
        self.tts.tts_to_file(text=text, speaker_wav=self.clone_voice_path, language="ar", file_path=self.out_path)
        return self.out_path

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 33.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 27.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 24.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 52.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 19.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of librosa to determine which version is compatible with other re

# Pipline

In [ ]:
class ImgToSpeechAR:
    def __init__(self):
        self.img2text = Img2Text("cuda")
        self.translator = Trans2Arabic()
        self.path = "/content/speaker.opus"
        self.T2S_ar = T2S_AR(self.path)

    def convert_Img_to_voice(self, img):
        img_des = self.img2text.run(img)
        self.img2text.cleanup()
        img_des_ar = self.translator.run(img_des)
        speech_ar_output = self.T2S_ar.run(img_des_ar)
        return speech_ar_output



# Gradio

In [ ]:
# import torch

# # Specify the device
# device = torch.device('cuda:0')

# # Clear cache on that specific device
# with torch.cuda.device(device):
#     torch.cuda.empty_cache()

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
!pip install gradio -q
import gradio as gr
# Initialize the class
pipline = ImgToSpeechAR()

# Define the Gradio interface
def gradio_interface(image):
    audio_path = pipline.convert_Img_to_voice(image)
    return audio_path

# Create the Gradio interface
iface = gr.Interface(
    fn=gradio_interface,

    inputs=gr.Image(type="pil", label="Upload Image"),
    outputs=gr.Audio(label="Generated Audio"),
    title="Speech to Image Converter",
    description="Upload an image, and the model will generate a speech description in Arabic.",
)

# Launch the interface
iface.launch(share=True, debug=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.7 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


config.json:   0%|          | 0.00/3.78k [00:00<?, ?B/s]

configuration_phi3_v.py:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-vision-instruct:
- configuration_phi3_v.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

 > Text splitted to sentences.
['تُصوّر الصورة لحظة مؤثرة على ملعب كرة القدم.', 'لاعب، يرتدي قميصاً أحمراً ساطعاً مع كلمة "ريوي" مكتوبة على الواجهة، مستلقي على الأرض، رأسه منخفض في ما يبدو أنه ضيق.', 'زملاؤه في الفريق، أيضاً يرتدون قمصاناً حمراء، يحيطون به، تعابيرهم مزيجاً من القلق والتعاطف.', 'زملاء اللاعب يركعون على جانبيه، أيديهم وضعت برفق على رأسه، يقدمون الراحة في هذه اللحظة من الضعف.', 'الخلفية مبهمة من الجمهور، وجوههم غير واضحة، ولكن حضورهم شهادة على أهمية اللحظة.', 'الصورة تذكير قوي بالمرات العاطفية العالية والمنخفضة التي تأتي مع الرياضة.']


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


 > Processing time: 21.163859605789185
 > Real-time factor: 0.47840507855572323


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 2018, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1567, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 8

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://41a38263cf5fa7c27d.gradio.live
